In [2]:
import numpy as np
import gensim
import logging
import networkx as nx
import pandas as pd

In [2]:
dismissed_list = pd.read_csv('data/dismissed.csv')


In [11]:
dismissed = pd.read_csv('data/dismissed_complete.csv')
non_dismissed = pd.read_csv('data/nodismissed_complete.csv')

#Delete random columns to have same number of dimissed and non dismissed

indexes_drop = np.arange(len(non_dismissed)-len(dismissed))
np.random.shuffle(indexes_drop)
non_dismissed = non_dismissed.drop(indexes_drop, axis = 0)

index = list(range(len(dismissed), len(dismissed)+len(non_dismissed)))

#non_dismissed.set_index(range(0,len(non_dismissed)+len(dismissed)))
all_authors = pd.concat((dismissed, non_dismissed))

index = range(0,len(all_authors))
all_authors['index'] = index
all_authors = all_authors.set_index('index')


/home/jogarrid/.virtualenvs/MLproject/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [23]:
network = {'Author': [], 'Connections':[], 'Connections_id':[],'Connections_label':[],'Label':[], 'Per Fired': []}

#id dictionary that associates name and id for the authors
ids = {}
for coauthors_ in dismissed['Co-authors']:
    coauthors = coauthors_.split(',')
    #"Clean list coauthors of weird characters"
    coauthors_clean = []
    for j in range(len(coauthors)-1):
        coauthors_clean.append(coauthors[j][2:len(coauthors[j])-1])
    j = len(coauthors) - 1
    coauthors_clean.append(coauthors[j][2:len(coauthors[j])-2])
    #Cleaning of list finished

    for i in range(len(coauthors_clean)):
        author = coauthors_clean[i]
        if(author not in network['Author']):
            network['Author'].append(author)
            ids[author] = len(network['Author']) - 1
            network['Connections'].append(coauthors_clean) #An author will always be connected to himself
            if(author in dismissed_list['AU'].unique()):
                network['Label'].append(1) #fired
            else:  #non fired
                network['Label'].append(0) #fired

        else: #Author already part of the network
            #in which index is the author in the network?
            found = False
            i = 0
            while(not found):
                if(network['Author'][i] == author):
                    found = True
                    ix = i
                i += 1
            #index found
            #For each coauthor: add it to the network of the main author
            for coauthor in coauthors_clean:
                if(coauthor not in network['Connections'][ix]):
                    network['Connections'][ix].append(coauthor)
            
   

In [29]:
#Add connections_id, id of authors in the connections
errors = 0
network['Connections_id'] = []
network['Connections_label'] = []
for i in range(len(network['Author'])):
    indexes = []
    labels = []
    for connection in network['Connections'][i]:
        #Find ix of connection
            ix = ids[connection]
            label = network['Label'][ix]
            indexes.append(ix)
            labels.append(label)

    network['Connections_id'].append(indexes)
    network['Connections_label'].append(labels)

#Add fields no of connections fired and no of connections not fired 
network['Connections_fired'] = []
network['Connections_nofired'] = []
for i in range(len(network['Author'])):
    fired = sum(network['Connections_label'][i])
    network['Connections_fired'].append(fired)
    network['Connections_nofired'].append(len(network['Connections_label'][i]) - fired)
    network['Per Fired'].append(fired/len(network['Connections_label'][i]))
                                
network_pd = pd.DataFrame(network)

"['Aydin Leyla', 'Aydin Aydin', 'Bulbul Baris', 'Ozkalkan Beliz', 'Ozaydin Cansu', 'Inaltekin Ezgi']"

In [1]:
import matplotlib.pyplot as plt

v_fired = network_pd[network_pd['Label']==1]
v_noFired = network_pd[network_pd['Label']==0]

#Box plot with quantiles 

plt.boxplot([v_noFired['Per Fired'], v_fired['Per Fired']])
plt.
network_pd['Per Fired'].describe()


SyntaxError: invalid syntax (<ipython-input-1-8e7798c7d65d>, line 12)

In [50]:
import scipy

#Correlation between label and percentage of connections fired
rho1, p_value1 = scipy.stats.pearsonr( np.array(network['Connections_fired'])/(np.array(network['Connections_nofired'])+np.array(network['Connections_fired'])), network['Label'])

#Correlation between label and absolute number of connections fired
rho2, p_value2 = scipy.stats.pearsonr(network['Label'], network['Connections_fired'])

#Correlation between label and absolute number of connections fired
rho3, p_value3 = scipy.stats.pearsonr(network['Label'], network['Connections_nofired'])

In [3]:
#Average number of connections
n_con = 0

for connections in network['Connections']:
    n_con += len(connections)

n_av = n_con / len(network['Author'])
print('The average number of connections is: ', n_av)

NameError: name 'network' is not defined

p_value3

In [49]:
#Next steps:

#See predictive power through taking into account no_fired, no_nofired

#Apply NLP to journal name

0.015963891438981217

SEE IF WE CAN PREDICT WHETHER SOMEONE GETS FIRED OR NOT BASED ON HOW MANY PEOPLE HE WAS WORKING WITH GOT FIRED